In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

In [ ]:
# Additional dependencies for this notebook
!pip install -q gem-suite

*Tantiya ng paggamit: 3 minuto sa isang Heron r2 processor (PAALALA: Ito ay tantiya lamang. Maaaring mag-iba ang inyong runtime.)*

## Background

Ipinapakita ng tutorial na ito kung paano irealisasyon ang Nishimori phase transition sa isang IBM&reg; quantum processor. Ang eksperimentong ito ay orihinal na inilarawan sa [*Realizing the Nishimori transition across the error threshold for constant-depth quantum circuits*](https://arxiv.org/abs/2309.02863).

Ang Nishimori phase transition ay tumutukoy sa transisyon sa pagitan ng short-range at long-range ordered phases sa random-bond Ising model. Sa isang quantum computer, ang long-range ordered phase ay lumalabas bilang isang estado kung saan ang mga qubit ay nag-entangle sa buong device. Ang lubhang naka-entangle na estadong ito ay inihahanda gamit ang *generation of entanglement by measurement* (GEM) protocol. Sa pamamagitan ng paggamit ng mid-circuit measurements, ang GEM protocol ay nakakapag-entangle ng mga qubit sa buong device gamit ang mga circuit na may constant depth lamang. Ginagamit ng tutorial na ito ang implementasyon ng GEM protocol mula sa [GEM Suite](https://github.com/qiskit-community/gem-suite) software package.

## Requirements

Bago magsimula sa tutorial na ito, siguraduhing mayroon kayong sumusunod na naka-install:

- Qiskit SDK v1.0 o mas bago, na may suporta sa [visualization](https://docs.quantum.ibm.com/api/qiskit/visualization)
- Qiskit Runtime v0.22 o mas bago ( `pip install qiskit-ibm-runtime` )
- GEM Suite ( `pip install gem-suite` )

## Setup

In [2]:
import matplotlib.pyplot as plt

from collections import defaultdict

from qiskit_ibm_runtime import QiskitRuntimeService

from qiskit.transpiler import generate_preset_pass_manager

from gem_suite import PlaquetteLattice
from gem_suite.experiments import GemExperiment

## Step 1: Map classical inputs to a quantum problem
Ang GEM protocol ay gumagana sa isang quantum processor na may qubit connectivity na inilarawan ng isang lattice. Ang mga IBM quantum processor ngayon ay gumagamit ng [heavy hex lattice](https://www.ibm.com/quantum/blog/heavy-hex-lattice). Ang mga qubit ng processor ay pinagsasama-sama sa mga *plaquette* batay sa kung aling unit cell ng lattice ang kanilang inookupahan. Dahil maaaring lumitaw ang isang qubit sa higit sa isang unit cell, ang mga plaquette ay hindi disjoint. Sa heavy hex lattice, ang isang plaquette ay naglalaman ng 12 qubit. Ang mga plaquette mismo ay bumubuo rin ng lattice, kung saan dalawang plaquette ay konektado kung may mga qubit silang pinagsasaluhan. Sa heavy hex lattice, ang mga kalapit na plaquette ay nagbabahagi ng 3 qubit.

Sa GEM Suite software package, ang pangunahing class para sa pagpapatupad ng GEM protocol ay ang `PlaquetteLattice`, na kumakatawan sa lattice ng mga plaquette (na iba sa heavy hex lattice). Ang isang `PlaquetteLattice` ay maaaring i-initialize mula sa isang qubit coupling map. Sa kasalukuyan, tanging ang heavy hex coupling maps lamang ang sinusuportahan.

Ang sumusunod na code cell ay nag-i-initialize ng plaquette lattice mula sa coupling map ng isang IBM quantum processor. Ang plaquette lattice ay hindi palaging sumasaklaw sa buong hardware. Halimbawa, ang `ibm_torino` ay may 133 kabuuang qubit, ngunit ang pinakamalaking plaquette lattice na umaangkop sa device ay gumagamit lamang ng 125 sa kanila, at binubuo ng kabuuang 18 plaquette. Katulad nito ay maaaring obserbahan para sa IBM Quantum&reg; devices na may iba't ibang bilang ng qubit.

In [ ]:
# QiskitRuntimeService.save_account(channel="ibm_quantum", token="<YOUR_API_KEYN>", overwrite=True, set_as_default=True)
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
plaquette_lattice = PlaquetteLattice.from_coupling_map(backend.coupling_map)

print(f"Number of qubits in backend: {backend.num_qubits}")
print(
    f"Number of qubits in plaquette lattice: {len(list(plaquette_lattice.qubits()))}"
)
print(f"Number of plaquettes: {len(list(plaquette_lattice.plaquettes()))}")

Number of qubits in backend: 133
Number of qubits in plaquette lattice: 125
Number of plaquettes: 18


You can visualize the plaquette lattice by generating a diagram of its graph representation. In the diagram, the plaquettes are represented by labeled hexagons, and two plaquettes are connected by an edge if they share qubits.

In [7]:
plaquette_lattice.draw_plaquettes()

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/625882a4-faeb-4d96-b441-c989f43c4dea-0.avif" alt="Output of the previous code cell" />

Maaari ninyong ivisualize ang plaquette lattice sa pamamagitan ng paggawa ng diagram ng graph representation nito. Sa diagram, ang mga plaquette ay kinakatawan ng mga naka-label na hexagon, at dalawang plaquette ay konektado ng isang edge kung may mga qubit silang pinagsasaluhan.

In [8]:
# Get a list of the plaquettes
plaquettes = list(plaquette_lattice.plaquettes())
# Display information about plaquette 0
plaquettes[0]

PyPlaquette(index=0, qubits=[0, 1, 2, 3, 4, 15, 16, 19, 20, 21, 22, 23], neighbors=[3, 1])

![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/625882a4-faeb-4d96-b441-c989f43c4dea-0.avif)

Maaari ninyong kunin ang impormasyon tungkol sa mga indibidwal na plaquette, tulad ng mga qubit na nilalaman nila, gamit ang `plaquettes` method.

In [9]:
plaquette_lattice.draw_qubits()

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/a19d63ce-3572-4081-a008-c1332fbbe303-0.avif" alt="Output of the previous code cell" />

In addition to the qubit labels and the edges indicating which qubits are connected, the diagram contains three additional pieces of information that are relevant to the GEM protocol:
- Each qubit is either shaded (gray) or unshaded. The shaded qubits are "site" qubits that represent the sites of the Ising model, and the unshaded qubits are "bond" qubits used to mediate interactions between the site qubits.
- Each site qubit is labeled either (A) or (B), indicating one of two roles a site qubit can play in the GEM protocol (the roles are explained later).
- Each edge is colored using one of six colors, thus partitioning the edges into six groups. This partitioning determines how two-qubit gates can be parallelized, as well as different scheduling patterns that are likely to incur different amounts of error on a noisy quantum processor. Because edges in a group are disjoint, a layer of two-qubit gates can be applied on those edges simultaneously. In fact, it is possible to partition the six colors into three groups of two colors such that the union of each group of two colors is still disjoint. Therefore, only three layers of two-qubit gates are needed to activate every edge. There are 12 ways to so partition the six colors, and each such partition yields a different 3-layer gate schedule.

Now that you have created a plaquette lattice, the next step is to initialize a `GemExperiment` object, passing both the plaquette lattice and the backend that you intend to run the experiment on. The `GemExperiment` class manages the actual implementation of the GEM protocol, including generating circuits, submitting jobs, and analyzing the data. The following code cell initializes the experiment class while restricting the plaquette lattice to only two of the plaquettes (21 qubits), reducing the size of the experiment to ensure that the noise in the hardware doesn't overwhelm the signal.

In [16]:
gem_exp = GemExperiment(plaquette_lattice.filter([9, 12]), backend=backend)

# visualize the plaquette lattice after filtering
plaquette_lattice.filter([9, 12]).draw_qubits()

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/02357c6e-5c83-4ac0-811d-22602d9f33d5-0.avif" alt="Output of the previous code cell" />

Maaari rin ninyong gumawa ng diagram ng mga pinagbabatayan na qubit na bumubuo sa plaquette lattice.

In [12]:
circuits = gem_exp.circuits()
print(f"Total number of circuits: {len(circuits)}")

Total number of circuits: 252


![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/a19d63ce-3572-4081-a008-c1332fbbe303-0.avif)

Bilang karagdagan sa mga qubit label at mga edge na nagsasaad kung aling mga qubit ang konektado, ang diagram ay naglalaman ng tatlong karagdagang piraso ng impormasyon na nauugnay sa GEM protocol:
- Ang bawat qubit ay may shading (gray) o walang shading. Ang mga shaded qubit ay "site" qubit na kumakatawan sa mga site ng Ising model, at ang mga unshaded qubit ay "bond" qubit na ginagamit upang magpagitna ng mga interaksyon sa pagitan ng mga site qubit.
- Ang bawat site qubit ay naka-label na (A) o (B), na nagsasaad ng isa sa dalawang tungkulin na maaaring gampanan ng isang site qubit sa GEM protocol (ang mga tungkulin ay ipaliliwanag sa ibang pagkakataon).
- Ang bawat edge ay may kulay gamit ang isa sa anim na kulay, kaya pinag-partition ang mga edge sa anim na grupo. Ang partition na ito ay tumutukoy kung paano maaaring i-parallelize ang mga two-qubit gate, gayundin ang iba't ibang mga scheduling pattern na malamang na magkaroon ng iba't ibang dami ng error sa isang maingay na quantum processor. Dahil ang mga edge sa isang grupo ay disjoint, ang isang layer ng two-qubit gates ay maaaring ilapat sa mga edge na iyon nang sabay-sabay. Sa katunayan, posible na hatiin ang anim na kulay sa tatlong grupo ng dalawang kulay na ang union ng bawat grupo ng dalawang kulay ay disjoint pa rin. Samakatuwid, tatlong layer lamang ng two-qubit gates ang kinakailangan upang ma-activate ang bawat edge. Mayroong 12 paraan upang i-partition ang anim na kulay sa ganitong paraan, at ang bawat partition na ito ay gumagawa ng ibang 3-layer gate schedule.

Ngayong nakagawa na kayo ng plaquette lattice, ang susunod na hakbang ay mag-initialize ng isang `GemExperiment` object, na nagpapasa ng parehong plaquette lattice at backend na balak ninyong patakbuhin ang eksperimento. Ang `GemExperiment` class ay namamahala sa aktwal na implementasyon ng GEM protocol, kasama ang paggawa ng mga circuit, pagsusumite ng mga job, at pagsusuri ng data. Ang sumusunod na code cell ay nag-i-initialize ng experiment class habang pinapaghigpit ang plaquette lattice sa dalawa lamang sa mga plaquette (21 qubit), binabawasan ang laki ng eksperimento upang masiguro na ang ingay sa hardware ay hindi makakalampas sa signal.

In [13]:
# Restrict experiment to the first scheduling pattern
gem_exp.set_experiment_options(schedule_idx=0)

# There are less circuits now
circuits = gem_exp.circuits()
print(f"Total number of circuits: {len(circuits)}")

# Print the RZZ angles swept over
print(f"RZZ angles:\n{gem_exp.parameters()}")

Total number of circuits: 21
RZZ angles:
[0.         0.07853982 0.15707963 0.23561945 0.31415927 0.39269908
 0.4712389  0.54977871 0.62831853 0.70685835 0.78539816 0.86393798
 0.9424778  1.02101761 1.09955743 1.17809725 1.25663706 1.33517688
 1.41371669 1.49225651 1.57079633]


![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/02357c6e-5c83-4ac0-811d-22602d9f33d5-0.avif)

Ang isang GEM protocol circuit ay binubuo gamit ang sumusunod na mga hakbang:
1. Ihanda ang all-$|+\rangle$ state sa pamamagitan ng paglalapat ng Hadamard gate sa bawat qubit.
2. Maglapat ng $R_{ZZ}$ gate sa pagitan ng bawat pares ng konektadong qubit. Ito ay maaaring makamit gamit ang 3 layer ng mga gate. Ang bawat $R_{ZZ}$ gate ay kumikilos sa isang site qubit at isang bond qubit. Kung ang site qubit ay naka-label na (B), ang anggulo ay nakatakda sa $\frac{\pi}{2}$. Kung ang site qubit ay naka-label na (A), ang anggulo ay pinapayagang mag-iba, na gumagawa ng iba't ibang mga circuit. Bilang default, ang hanay ng mga anggulo ay nakatakda sa 21 pantay na espasyong mga punto sa pagitan ng $0$ at $\frac{\pi}{2}$, inclusive.
3. Sukatin ang bawat bond qubit sa Pauli $X$ basis. Dahil ang mga qubit ay sinusukat sa Pauli $Z$ basis, ito ay maaaring magawa sa pamamagitan ng paglalapat ng Hadamard gate bago sukatin ang qubit.

Tandaan na ang papel na binanggit sa panimula ng tutorial na ito ay gumagamit ng ibang convention para sa $R_{ZZ}$ angle, na naiiba sa convention na ginamit sa tutorial na ito ng factor na 2.

Sa hakbang 3, tanging ang mga bond qubit lamang ang sinusukat. Upang maintindihan kung anong estado ang nananatili sa mga site qubit, nakakatulong na isaalang-alang ang kaso na ang $R_{ZZ}$ angle na inilapat sa mga site qubit (A) sa hakbang 2 ay katumbas ng $\frac{\pi}{2}$. Sa kasong ito, ang mga site qubit ay naiiwanan sa isang lubhang naka-entangle na estado na katulad ng GHZ state,

$$
\lvert \text{GHZ} \rangle = \lvert 00 \cdots 00 \rangle + \lvert 11 \cdots 11 \rangle.
$$

Dahil sa randomness sa mga measurement outcome, ang aktwal na estado ng mga site qubit ay maaaring ibang estado na may long-range order, halimbawa, $\lvert 00110 \rangle + \lvert 11001 \rangle$. Gayunpaman, ang GHZ state ay maaaring mabawi sa pamamagitan ng paglalapat ng decoding operation batay sa mga measurement outcome. Kapag ang $R_{ZZ}$ angle ay na-tune down mula sa $\frac{\pi}{2}$, ang long-range order ay maaari pa ring mabawi hanggang sa isang critical angle, na sa kawalan ng ingay, ay humigit-kumulang $0.3 \pi$. Sa ibaba ng anggulo na ito, ang resultang estado ay hindi na nagpapakita ng long-range entanglement. Ang transisyon na ito sa pagitan ng presensya at kawalan ng long-range order ay ang Nishimori phase transition.

Sa paglalarawang nasa itaas, ang mga site qubit ay hindi sinukat, at ang decoding operation ay maaaring isagawa sa pamamagitan ng paglalapat ng mga quantum gate. Sa eksperimento gaya ng ipinatupad sa GEM suite, na sinusundan ng tutorial na ito, ang mga site qubit ay sinusukat, at ang decoding operation ay inilalapat sa isang classical post-processing step.

Sa paglalarawang nasa itaas, ang decoding operation ay maaaring isagawa sa pamamagitan ng paglalapat ng mga quantum gate sa mga site qubit upang mabawi ang quantum state. Gayunpaman, kung ang layunin ay agarang sukatin ang estado, halimbawa, para sa mga layunin ng characterization, ang mga site qubit ay sinusukat kasama ng mga bond qubit, at ang decoding operation ay maaaring ilapat sa isang classical post-processing step. Ito ang paraan ng pagpapatupad ng eksperimento sa GEM suite, na sinusundan ng tutorial na ito.

Bilang karagdagan sa pag-asa sa $R_{ZZ}$ angle sa hakbang 2, na bilang default ay sumasalakay sa 21 values, ang GEM protocol circuit ay umaasa rin sa scheduling pattern na ginamit upang ipatupad ang 3 layer ng $R_{ZZ}$ gates. Tulad ng tinalakay dati, mayroong 12 gayong scheduling pattern. Samakatuwid, ang kabuuang bilang ng mga circuit sa eksperimento ay $21 \times 12 = 252$.

Ang mga circuit ng eksperimento ay maaaring i-generate gamit ang `circuits` method ng `GemExperiment` class.

In [14]:
# Get the circuit at index 5
circuit = circuits[5]
# Remove the final measurements to ease visualization
circuit.remove_final_measurements()
# Draw the circuit
circuit.draw("mpl", fold=-1, scale=0.5)

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/fd57d483-c70b-4ad5-b309-15750ad38bac-0.avif" alt="Output of the previous code cell" />

## Step 2: Optimize problem for quantum hardware execution

Transpiling quantum circuits for execution on hardware typically involves a [number of stages](/docs/guides/transpiler-stages). Typically, the stages that incur the most computational overhead are choosing the qubit layout, routing the two-qubit gates to conform to the qubit connectivity of the hardware, and optimizing the circuit to minimize its gate count and depth. In the GEM protocol, the layout and routing stages are unnecessary because the hardware connectivity is already incorporated into the design of the protocol. The circuits already have a qubit layout, and the two-qubit gates are already mapped onto native connections. Furthermore, in order to preserve the structure of the circuit as the $R_{ZZ}$ angle is varied, only very basic circuit optimization should be performed.

The `GemExperiment` class transparently transpiles circuits when executing the experiment. The layout and routing stages are already overridden by default to do nothing, and circuit optimization is performed at a level that only optimizes single-qubit gates. However, you can override or pass additional options using the `set_transpile_options` method. For the sake of visualization, the following code cell manually transpiles the circuit displayed previously, and draws the transpiled circuit.

In [15]:
# Demonstrate setting transpile options
gem_exp.set_transpile_options(
    optimization_level=1  # This is the default optimization level
)
pass_manager = generate_preset_pass_manager(
    backend=backend,
    initial_layout=list(gem_exp.physical_qubits),
    **dict(gem_exp.transpile_options),
)
transpiled = pass_manager.run(circuit)
transpiled.draw("mpl", idle_wires=False, fold=-1, scale=0.5)

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/e9b99d48-8d33-46b5-bff5-480ab1c1c1f2-0.avif" alt="Output of the previous code cell" />

Para sa mga layunin ng tutorial na ito, sapat na na isaalang-alang ang isang scheduling pattern lamang. Ang sumusunod na code cell ay pinaghihigpit ang eksperimento sa unang scheduling pattern. Bilang resulta, ang eksperimento ay may 21 circuits lamang, isa para sa bawat $R_{ZZ}$ angle na sinasalakay.

In [10]:
exp_data = gem_exp.run(shots=10_000)

To wait for the results, call the `block_for_results` method of the `ExperimentData` object. This call will cause the interpreter to hang until the jobs are finished.

In [11]:
exp_data.block_for_results()

ExperimentData(GemExperiment, d0d5880a-34c1-4aab-a7b6-c4f58516bc03, job_ids=['cwg12ptmptp00082khhg'], metadata=<5 items>, figure_names=['two_point_correlation.svg', 'normalized_variance.svg', 'plaquette_ops.svg', 'bond_ops.svg'])

Ang sumusunod na code cell ay gumaguhit ng diagram ng circuit sa index 5. Upang bawasan ang laki ng diagram, ang mga measurement gate sa dulo ng circuit ay inalis.

In [ ]:
def magnetization_distribution(
    counts_dict: dict[str, int],
) -> dict[str, float]:
    """Compute magnetization distribution from counts dictionary."""
    # Construct dictionary from magnetization to count
    mag_dist = defaultdict(float)
    for bitstring, count in counts_dict.items():
        mag = bitstring.count("0") - bitstring.count("1")
        mag_dist[mag] += count
    # Normalize
    shots = sum(counts_dict.values())
    for mag in mag_dist:
        mag_dist[mag] /= shots
    return mag_dist


# Get counts dictionaries with and without decoding
data = exp_data.data()
# Get the last data point, which is at the angle for the GHZ state
raw_counts = data[-1]["counts"]
# Without decoding
site_indices = [
    i for i, q in enumerate(gem_exp.plaquettes.qubits()) if q.role == "Site"
]
site_raw_counts = defaultdict(int)
for key, val in raw_counts.items():
    site_str = "".join(key[-1 - i] for i in site_indices)
    site_raw_counts[site_str] += val
# With decoding
_, site_decoded_counts = gem_exp.plaquettes.decode_outcomes(
    raw_counts, return_counts=True
)

# Compute magnetization distribution
raw_magnetization = magnetization_distribution(site_raw_counts)
decoded_magnetization = magnetization_distribution(site_decoded_counts)

# Plot
plt.bar(*zip(*raw_magnetization.items()), label="raw")
plt.bar(*zip(*decoded_magnetization.items()), label="decoded", width=0.3)
plt.legend()
plt.xlabel("Magnetization")
plt.ylabel("Frequency")
plt.title("Magnetization distribution with and without decoding")

Text(0.5, 1.0, 'Magnetization distribution with and without decoding')

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/8ead3582-16df-4616-836c-bdce867ad6b8-1.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/fd57d483-c70b-4ad5-b309-15750ad38bac-0.avif)

## Step 2: Optimize problem for quantum hardware execution
Ang pag-transpile ng mga quantum circuit para sa pagsasagawa sa hardware ay karaniwang nagsasangkot ng [ilang mga yugto](/guides/transpiler-stages). Kadalasan, ang mga yugtong may pinakamataas na computational overhead ay ang pagpili ng qubit layout, pag-route ng mga two-qubit gate upang umangkop sa qubit connectivity ng hardware, at pag-optimize ng circuit upang mabawasan ang gate count at depth nito. Sa GEM protocol, ang layout at routing stages ay hindi kinakailangan dahil ang hardware connectivity ay naisama na sa disenyo ng protocol. Ang mga circuit ay mayroon nang qubit layout, at ang mga two-qubit gate ay na-map na sa mga native connection. Higit pa rito, upang mapanatili ang istraktura ng circuit habang nag-iiba ang $R_{ZZ}$ angle, ang napaka-basic na circuit optimization lamang ang dapat isagawa.

Ang `GemExperiment` class ay transparent na nag-transpile ng mga circuit kapag isinasagawa ang eksperimento. Ang layout at routing stages ay na-override na bilang default upang walang gawin, at ang circuit optimization ay isinasagawa sa isang antas na nag-o-optimize lamang ng mga single-qubit gate. Gayunpaman, maaari ninyong i-override o magpasa ng karagdagang mga opsyon gamit ang `set_transpile_options` method. Para sa visualization, ang sumusunod na code cell ay manually nag-transpile ng circuit na ipinakita dati, at gumuhit ng transpiled circuit.

In [13]:
exp_data.figure("two_point_correlation")

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/4ecb25c8-e572-49af-a879-9943039db131-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/e9b99d48-8d33-46b5-bff5-480ab1c1c1f2-0.avif)

## Step 3: Execute using Qiskit primitives
Upang maisagawa ang mga GEM protocol circuit sa hardware, tawagan ang `run` method ng `GemExperiment` object. Maaari ninyong tukuyin ang bilang ng mga shot na nais ninyong i-sample mula sa bawat circuit. Ang `run` method ay nagbabalik ng isang [ExperimentData](https://qiskit-community.github.io/qiskit-experiments/stubs/qiskit_experiments.framework.ExperimentData.html) object na dapat ninyong i-save sa isang variable. Tandaan na ang `run` method ay nagsusumite lamang ng mga job nang hindi naghihintay na matapos ang mga ito, kaya ito ay isang non-blocking call.

In [14]:
exp_data.figure("normalized_variance")

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/2b351d68-3924-445a-94ef-047b16214e8a-0.avif" alt="Output of the previous code cell" />

Upang maghintay para sa mga resulta, tawagan ang `block_for_results` method ng `ExperimentData` object. Ang tawag na ito ay magiging sanhi na ang interpreter ay magsususpende hanggang sa matapos ang mga job.

In [15]:
gem_exp = GemExperiment(
    plaquette_lattice.filter(range(3, 9)), backend=backend
)
gem_exp.set_experiment_options(schedule_idx=0)
exp_data = gem_exp.run(shots=10_000)
exp_data.block_for_results()
exp_data.figure("normalized_variance")

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/08581c09-a6a5-4a56-9fc4-abf22b063c6a-0.avif" alt="Output of the previous code cell" />

In [16]:
gem_exp = GemExperiment(plaquette_lattice, backend=backend)
gem_exp.set_experiment_options(schedule_idx=0)
exp_data = gem_exp.run(shots=10_000)
exp_data.block_for_results()
exp_data.figure("normalized_variance")

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/37e9a4cd-6efb-4ade-ad09-8139db9d58e9-0.avif" alt="Output of the previous code cell" />

## Step 4: Post-process and return result in desired classical format
Sa isang $R_{ZZ}$ angle na $\frac{\pi}{2}$, ang decoded state ay magiging GHZ state sa kawalan ng ingay. Ang long-range order ng GHZ state ay maaaring ivisualize sa pamamagitan ng pag-plot ng magnetization ng mga sinukatang bitstring. Ang magnetization $M$ ay tinukoy bilang kabuuan ng mga single-qubit Pauli $Z$ operator,
$$
M = \sum_{j=1}^N Z_j,
$$
kung saan ang $N$ ay ang bilang ng mga site qubit. Ang halaga nito para sa isang bitstring ay katumbas ng pagkakaiba sa pagitan ng bilang ng mga zero at bilang ng mga one. Ang pagsukat ng GHZ state ay nagbubunga ng all zeros state o all ones state na may pantay na probability, kaya ang magnetization ay magiging $+N$ kalahati ng oras at $-N$ ang kalahati ng oras. Sa presensya ng mga error dahil sa ingay, ang ibang mga value ay lilitaw din, ngunit kung ang ingay ay hindi masyadong malaki, ang distribusyon ay nakasentro pa rin malapit sa $+N$ at $-N$.

Para sa mga raw bitstring bago ang decoding, ang distribusyon ng magnetization ay magiging katumbas ng mga uniformly random bitstring, sa kawalan ng ingay.

Ang sumusunod na code cell ay nag-plot ng magnetization ng mga raw bitstring at decoded bitstring sa $R_{ZZ}$ angle na $\frac{\pi}{2}$.